In [ ]:
# Example - https://github.com/cornelliusyudhawijaya/Churn_Causality_Analysis/blob/main/Causal%20Analysis%20Do%20Why.ipynb
    
import numpy as np
import pandas as pd

from dowhy import CausalModel
import dowhy.datasets 

# Avoid printing dataconversion warnings from sklearn and numpy
import warnings
from sklearn.exceptions import DataConversionWarning
warnings.filterwarnings('ignore')

In [ ]:
from sklearn import preprocessing
data = pd.read_csv('../../../data/MEPS/h181.csv')

# ## Drop NULL values
data = data.dropna()


data = data.rename(columns = {'FTSTU53X' : 'FTSTU', 'ACTDTY53' : 'ACTDTY', 'HONRDC53' : 'HONRDC', 'RTHLTH53' : 'RTHLTH',
                              'MNHLTH53' : 'MNHLTH', 'CHBRON53' : 'CHBRON', 'JTPAIN53' : 'JTPAIN', 'PREGNT53' : 'PREGNT',
                              'WLKLIM53' : 'WLKLIM', 'ACTLIM53' : 'ACTLIM', 'SOCLIM53' : 'SOCLIM', 'COGLIM53' : 'COGLIM',
                              'EMPST53' : 'EMPST', 'REGION53' : 'REGION', 'MARRY53X' : 'MARRY', 'AGE53X' : 'AGE',
                              'POVCAT15' : 'POVCAT', 'INSCOV15' : 'INSCOV'})


data = data[data['PANEL'] == 20]
data = data[data['REGION'] >= 0] # remove values -1
data = data[data['AGE'] >= 0] # remove values -1
data = data[data['MARRY'] >= 0] # remove values -1, -7, -8, -9
data = data[data['ASTHDX'] >= 0] # remove values -1, -7, -8, -9
data = data[(data[['FTSTU','ACTDTY','HONRDC','RTHLTH','MNHLTH','HIBPDX','CHDDX','ANGIDX','EDUCYR','HIDEG',
                             'MIDX','OHRTDX','STRKDX','EMPHDX','CHBRON','CHOLDX','CANCERDX','DIABDX',
                             'JTPAIN','ARTHDX','ARTHTYPE','ASTHDX','ADHDADDX','PREGNT','WLKLIM',
                             'ACTLIM','SOCLIM','COGLIM','DFHEAR42','DFSEE42','ADSMOK42',
                             'PHQ242','EMPST','POVCAT','INSCOV']] >= -1).all(1)]

# ## Change symbolics to numerics
data['RACEV2X'] = np.where((data['HISPANX'] == 2 ) & (data['RACEV2X'] == 1), 1, data['RACEV2X'])
data['RACEV2X'] = np.where(data['RACEV2X'] != 1 , 0, data['RACEV2X'])
data = data.rename(columns={"RACEV2X" : "RACE"})
# data['UTILIZATION'] = np.where(data['UTILIZATION'] >= 10, 1, 0)



def utilization(row):
        return row['OBTOTV15'] + row['OPTOTV15'] + row['ERTOT15'] + row['IPNGTD15'] + row['HHTOTD15']

data['TOTEXP15'] = data.apply(lambda row: utilization(row), axis=1)
lessE = data['TOTEXP15'] < 10.0
data.loc[lessE,'TOTEXP15'] = 0.0
moreE = data['TOTEXP15'] >= 10.0
data.loc[moreE,'TOTEXP15'] = 1.0

data = data.rename(columns = {'TOTEXP15' : 'UTILIZATION'})

data = data[['REGION','AGE','SEX','RACE','MARRY',
                                 'FTSTU','ACTDTY','HONRDC','RTHLTH','MNHLTH','HIBPDX','CHDDX','ANGIDX',
                                 'MIDX','OHRTDX','STRKDX','EMPHDX','CHBRON','CHOLDX','CANCERDX','DIABDX',
                                 'JTPAIN','ARTHDX','ARTHTYPE','ASTHDX','ADHDADDX','PREGNT','WLKLIM',
                                 'ACTLIM','SOCLIM','COGLIM','DFHEAR42','DFSEE42', 'ADSMOK42',
                                 'PCS42','MCS42','K6SUM42','PHQ242','EMPST','POVCAT','INSCOV','UTILIZATION', 'PERWT15F']]

data = data.rename(columns={"UTILIZATION": "Probability","RACE" : "race"})
protected_attribute = 'race'


from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
data = pd.DataFrame(scaler.fit_transform(data),columns = data.columns)

print(data.head(5))

In [ ]:
causal_graph = """
digraph {
SEX;
AGE;
race;
MARRY;
REGION;
Probability;
U[label="Unobserved Confounders"];
SEX -> Probability;
race -> REGION; REGION -> Probability;
race -> Probability;
AGE -> Probability;
SEX -> MARRY; MARRY -> Probability;
U->SEX;U->race;U->Probability;
}
"""

## dowhy works for binary inputs only


data['Probability'] = np.where(data['Probability'] == 0, False, True)
data[protected_attribute] = np.where(data[protected_attribute] == 1, True, False)

In [ ]:
# With graph
model=CausalModel(
        data = data,
        treatment=protected_attribute,
        outcome="Probability",
        graph=causal_graph.replace("\n", " ")
        )
model.view_model()

In [ ]:
identified_estimand = model.identify_effect(proceed_when_unidentifiable=True)
print(identified_estimand)

In [ ]:
causal_estimate = model.estimate_effect(identified_estimand,
        method_name="backdoor.propensity_score_stratification")
print(causal_estimate)
print("Causal Estimate is " + str(causal_estimate.value))

In [ ]:
# Causal effect on the control group (ATC)
causal_estimate_att = model.estimate_effect(identified_estimand,
        method_name="backdoor.propensity_score_stratification",
        target_units = "atc")
print(causal_estimate_att)
print("Causal Estimate is " + str(causal_estimate_att.value))

# Add Random Common Cause

In [ ]:
refutel = model.refute_estimate(identified_estimand,causal_estimate, "random_common_cause")
print(refutel)

# Replace Treatment with Placebo

In [ ]:
refutel = model.refute_estimate(identified_estimand,causal_estimate,method_name="placebo_treatment_refuter",placebo_type="permute")
print(refutel)

# Remove Random Subset of Data

In [ ]:
refutel = model.refute_estimate(identified_estimand,causal_estimate, "data_subset_refuter")
print(refutel)